In [ ]:
import numpy as np
import cv2
import pandas as pd

import pickle
from matplotlib import pyplot as pyplot
import os

1. Read Training Images

In [ ]:
image_dataset=pd.DataFrame(columns=["Image_Name","Pixel_Values"])

img_path = "train_images/"
for img_name in sorted(os.listdir(img_path)):
    print(img_name)

    #df=pd.DataFrame()
    #df["Image_Name"] = img_name

    input_img=cv2.imread(img_path+img_name)

    #Check if the image is RGB or grey and convert to grey if they are RGB
    if input_img.ndim==3 and input_img.shape[-1]==3:
        img=cv2.cvtColor(input_img,cv2.COLOR_RGB2GRAY)
    elif input_img.dim==2:
        img=input_img
    else:
        raise Exception("Only work with RGB and grayscale images")
    
    ###################################################################################################################################
    # Add data into the dataframe
    # The first feature is the pixel values

    pixel_values=img.reshape(-1)
    #df["Pixel_Values"] = pixel_values
    new_row={"Image_Name":img_name,"Pixel_Values":pixel_values}
    image_dataset=pd.concat([image_dataset,pd.DataFrame([new_row])],ignore_index=True)

2. Generate Gabor Features

In [ ]:
gabor_df=pd.DataFrame()
gabor_df["Image_Name"]=''
for index,row in image_dataset.iterrows():
    
    img_path = "train_images/"
    img=cv2.imread(img_path+row["Image_Name"])
    
    num=1
    kernels=[]
    filtered_img_arr=[row["Image_Name"]]

    for theta in range(2):
        theta = theta/4. *np.pi
        for sigma in (1,3):
            for lamda in np.arange(0,np.pi,np.pi/4):
                for gamma in (0.005,0.5):
                    gabor_label = 'Gabor'+str(num) #Label Gabor columns as Gabor1, Gabor2...
                    if gabor_label not in gabor_df.columns:
                        gabor_df[gabor_label]=''
                    ksize=9
                    kernel = cv2.getGaborKernel((ksize,ksize),sigma,theta,lamda,gamma,0,ktype=cv2.CV_32F)
                    kernels.append(kernel)
                    
                    #Filter image and add values to the column
                    fimg=cv2.filter2D(img,cv2.CV_8UC3,kernel)
                    filtered_img=fimg.reshape(-1)
                    #row[gabor_label]=filtered_img
                    filtered_img_arr.append(filtered_img)
                    print(gabor_label,': theta=',theta,': sigma=',sigma,': lamda=',lamda,': gamma=',gamma)
                    num+=1

    gabor_df.loc[len(gabor_df)]=filtered_img_arr

image_dataset=image_dataset.merge(gabor_df,how='left',on='Image_Name')


2. Generate SIFT features